# NVIDIA GPU Setup Guide for WSL in CodexContinue

This notebook provides a step-by-step guide for diagnosing and fixing NVIDIA GPU support in Windows Subsystem for Linux (WSL) for the CodexContinue project. The guide focuses on resolving common issues with missing libraries and ensuring proper Docker integration with GPU support.

## Import Required Libraries

First, let's import the necessary Python libraries for system checks and diagnostics.

In [ ]:
# Ensure IPython is installed
%pip install ipython

import os
import subprocess
import json
import glob
import sys

# For prettier output
from IPython.display import display, Markdown

## Check NVIDIA Libraries in WSL

Let's check for the presence of NVIDIA libraries in the standard WSL location (`/usr/lib/wsl/lib/libnvidia-ml.so`).

In [ ]:
def check_nvidia_libraries():
    wsl_lib_path = "/usr/lib/wsl/lib/libnvidia-ml.so"
    if os.path.exists(wsl_lib_path):
        display(Markdown(f"✅ Found NVIDIA libraries in WSL driver location: `{wsl_lib_path}`"))
    else:
        display(Markdown(f"❌ Missing NVIDIA libraries in standard WSL location: `{wsl_lib_path}`"))
        
    # Check LD_LIBRARY_PATH
    ld_library_path = os.environ.get('LD_LIBRARY_PATH', '')
    display(Markdown(f"Current LD_LIBRARY_PATH: `{ld_library_path}`"))

check_nvidia_libraries()

## Verify `nvidia-smi` Output

Now, let's run the `nvidia-smi` command to verify GPU status.

In [ ]:
def run_nvidia_smi():
    try:
        result = subprocess.run(['which', 'nvidia-smi'], capture_output=True, text=True, check=False)
        if result.returncode == 0:
            display(Markdown(f"Found nvidia-smi at: `{result.stdout.strip()}`"))
            
            # Run nvidia-smi
            nvidia_smi_output = subprocess.run(['nvidia-smi'], capture_output=True, text=True, check=False)
            if nvidia_smi_output.returncode == 0:
                display(Markdown("### nvidia-smi output:"))
                print(nvidia_smi_output.stdout)
                return True
            else:
                display(Markdown(f"❌ Error running nvidia-smi: {nvidia_smi_output.stderr}"))
        else:
            display(Markdown("❌ nvidia-smi command not found"))
    except Exception as e:
        display(Markdown(f"❌ Error: {str(e)}"))
    return False

ran_successfully = run_nvidia_smi()

## Search for NVIDIA Libraries

Let's search for `libnvidia-ml.so` in common library paths and display the results.

In [ ]:
def search_nvidia_libraries():
    display(Markdown("Looking for libnvidia-ml.so in common paths:"))
    
    search_paths = [
        "/usr/lib",
        "/usr/lib/x86_64-linux-gnu",
        "/usr/lib/wsl/lib",
        "/usr/local/cuda*/targets/x86_64-linux/lib"
    ]
    
    found_libs = []
    
    for path in search_paths:
        # Handle glob patterns
        if '*' in path:
            matching_dirs = glob.glob(path)
            for dir in matching_dirs:
                lib_file = os.path.join(dir, "libnvidia-ml.so")
                if os.path.exists(lib_file):
                    found_libs.append(lib_file)
                # Check for stub files
                stub_path = os.path.join(dir, "stubs/libnvidia-ml.so")
                if os.path.exists(stub_path):
                    found_libs.append(stub_path + " (stub)")
        else:
            lib_file = os.path.join(path, "libnvidia-ml.so")
            if os.path.exists(lib_file):
                found_libs.append(lib_file)
    
    if found_libs:
        for lib in found_libs:
            display(Markdown(f"- Found `{lib}`"))
    else:
        display(Markdown("❌ No NVIDIA libraries found in common paths"))

search_nvidia_libraries()

## Check NVIDIA Container Toolkit Configuration

Let's read and parse the Docker daemon configuration file to verify NVIDIA runtime settings.

In [ ]:
def check_docker_nvidia_config():
    docker_config_path = "/etc/docker/daemon.json"
    
    if os.path.exists(docker_config_path):
        try:
            with open(docker_config_path, 'r') as f:
                config = json.load(f)
                
            display(Markdown("### Docker daemon configuration:"))
            print(json.dumps(config, indent=4))
            
            if 'runtimes' in config and 'nvidia' in config['runtimes']:
                display(Markdown("✅ NVIDIA Container Runtime is configured"))
                return True
            else:
                display(Markdown("❌ NVIDIA Container Runtime is not configured"))
        except Exception as e:
            display(Markdown(f"Error reading Docker configuration: {str(e)}"))
    else:
        display(Markdown(f"❌ Docker daemon configuration file not found at {docker_config_path}"))
    
    return False

docker_configured = check_docker_nvidia_config()

## Test Docker GPU Capability

Now let's run a Docker command to check for GPU support.

In [ ]:
def test_docker_gpu():
    display(Markdown("### Testing Docker with GPU support:"))
    
    # First check docker info for nvidia runtime
    try:
        docker_info = subprocess.run(['docker', 'info'], capture_output=True, text=True, check=False)
        if 'nvidia' in docker_info.stdout and 'Runtimes' in docker_info.stdout:
            display(Markdown("✅ Docker info shows nvidia runtime is available"))
        else:
            display(Markdown("❌ Docker info does not show nvidia runtime"))
            print("Docker runtimes found:")
            for line in docker_info.stdout.splitlines():
                if 'Runtimes' in line:
                    print(line)
    except Exception as e:
        display(Markdown(f"Error checking Docker info: {str(e)}"))
    
    # Try running a container with GPU access
    try:
        display(Markdown("### Testing GPU access with a CUDA container:"))
        display(Markdown("Running: `docker run --rm --gpus all nvidia/cuda:11.6.2-base-ubuntu20.04 nvidia-smi`"))
        
        # Run the command but don't capture output as it might be large
        display(Markdown("This might take a moment to pull the container image if it's not already cached..."))
        result = subprocess.run(
            ['docker', 'run', '--rm', '--gpus', 'all', 'nvidia/cuda:11.6.2-base-ubuntu20.04', 'nvidia-smi'],
            capture_output=True, text=True, check=False
        )
        
        if result.returncode == 0:
            display(Markdown("✅ Successfully ran nvidia-smi in a Docker container with GPU access"))
            print(result.stdout)
            return True
        else:
            display(Markdown(f"❌ Failed to run nvidia-smi in Docker: {result.stderr}"))
    except Exception as e:
        display(Markdown(f"Error testing Docker GPU support: {str(e)}"))
    
    return False

# Only run if the user wants to test Docker
test_docker = input("Do you want to test Docker with GPU support? (y/n): ")
if test_docker.lower() == 'y':
    docker_test_successful = test_docker_gpu()
else:
    display(Markdown("Skipping Docker GPU test"))
    docker_test_successful = None

## Provide Fix Suggestions

Based on the diagnostic results, here are actionable suggestions to fix common issues.

In [ ]:
def provide_fixes():
    display(Markdown("## Diagnostic Summary and Fix Suggestions"))
    
    # Check if NVIDIA libraries exist in /usr/lib/wsl/lib
    wsl_lib_path = "/usr/lib/wsl/lib/libnvidia-ml.so"
    wsl_lib_exists = os.path.exists(wsl_lib_path)
    
    # Check if we found nvidia-smi earlier
    nvidia_smi_runs = ran_successfully
    
    # Check Docker configuration
    docker_configured_correctly = docker_configured
    
    # Print summary
    display(Markdown("### Status Summary:"))
    display(Markdown(f"- NVIDIA libraries in WSL path: {'✅ Found' if wsl_lib_exists else '❌ Missing'}"))
    display(Markdown(f"- nvidia-smi runs successfully: {'✅ Yes' if nvidia_smi_runs else '❌ No'}"))
    display(Markdown(f"- Docker NVIDIA runtime configured: {'✅ Yes' if docker_configured_correctly else '❌ No'}"))
    
    # Generate fix suggestions
    display(Markdown("### Fix Suggestions:"))
    
    fixes = []
    
    if not wsl_lib_exists and nvidia_smi_runs:
        # We have NVIDIA drivers but missing the symbolic link
        fixes.append("**Missing Library Symbolic Link**:"
                  "\n1. Run the fix script: `sudo ./scripts/fix-nvidia-wsl-libs.sh`"
                  "\n2. This will create the symbolic link from an existing library to the standard WSL location")
    
    if not nvidia_smi_runs:
        fixes.append("**NVIDIA Driver Issues**:"
                  "\n1. Verify you have the NVIDIA driver for WSL installed on Windows (not in WSL)"
                  "\n2. Install from: https://developer.nvidia.com/cuda/wsl"
                  "\n3. Restart your Windows system after installation"
                  "\n4. Make sure your GPU is CUDA-capable and supported by WSL2")
    
    if not docker_configured_correctly:
        fixes.append("**Docker NVIDIA Runtime Configuration**:"
                  "\n1. Install NVIDIA Container Toolkit: `sudo apt-get install -y nvidia-container-toolkit`"
                  "\n2. Configure Docker: `sudo nvidia-ctk runtime configure --runtime=docker`"
                  "\n3. Restart Docker: `sudo systemctl restart docker`")
    
    if not fixes:
        display(Markdown("✅ **Congratulations!** Your NVIDIA GPU appears to be properly configured for WSL and Docker."))
        display(Markdown("You should now be able to use GPU acceleration with Ollama in the CodexContinue project."))
        display(Markdown("Run the following to start with GPU support:"))
        display(Markdown("`docker compose -f docker-compose.yml up`"))
    else:
        for i, fix in enumerate(fixes):
            display(Markdown(f"#### Fix {i+1}:\n{fix}"))
    
    # Add CodexContinue specific notes
    display(Markdown("### CodexContinue Project Notes:"))
    display(Markdown("1. After fixing GPU issues, test the setup with Ollama:"))
    display(Markdown("   ```bash\n   ./scripts/start-ollama-wsl.sh\n   ```"))
    display(Markdown("2. If problems persist in CodexContinue, use the CPU-only configuration:"))
    display(Markdown("   ```bash\n   docker compose -f docker-compose.yml -f docker-compose.macos.yml up\n   ```"))

provide_fixes()

## Troubleshooting Ollama with GPU Support

Ollama can be run in different ways with GPU support in WSL. Let's explore the options and common issues.

In [ ]:
def check_ollama_processes():
    display(Markdown("### Checking for running Ollama processes:"))
    
    # Check if Ollama is running as a standalone process
    try:
        result = subprocess.run(['pgrep', '-f', 'ollama'], capture_output=True, text=True, check=False)
        if result.stdout.strip():
            pids = result.stdout.strip().split('\n')
            display(Markdown(f"⚠️ Found Ollama running as standalone process with PID(s): {', '.join(pids)}"))
            
            # Check if it's listening on port 11434
            port_check = subprocess.run(
                ['sudo', 'lsof', '-i', ':11434'], 
                capture_output=True, text=True, check=False
            )
            if port_check.stdout and 'ollama' in port_check.stdout:
                display(Markdown("⚠️ Ollama is using port 11434, which will conflict with Docker containers"))
                display(Markdown("To stop the standalone Ollama service:"))
                display(Markdown("```bash\nsudo pkill -f ollama\n```"))
            return True
        else:
            display(Markdown("✅ No standalone Ollama processes found"))
            return False
    except Exception as e:
        display(Markdown(f"Error checking Ollama processes: {str(e)}"))
        return False
    
    # Check for Docker containers running Ollama
    try:
        docker_check = subprocess.run(
            ['docker', 'ps', '--filter', 'name=ollama'], 
            capture_output=True, text=True, check=False
        )
        if 'ollama' in docker_check.stdout:
            display(Markdown("⚠️ Found Ollama running in Docker container"))
            display(Markdown("```\n" + docker_check.stdout + "\n```"))
            display(Markdown("To stop Ollama Docker containers:"))
            display(Markdown("```bash\ndocker stop $(docker ps -q --filter name=ollama)\n```"))
            return True
        else:
            display(Markdown("✅ No Ollama Docker containers found"))
            return False
    except Exception as e:
        display(Markdown(f"Error checking Docker: {str(e)}"))
        return False

ollama_running = check_ollama_processes()

### Ollama GPU Support Options

There are several ways to run Ollama with GPU support in WSL:

1. **Standalone Ollama with GPU** - Install Ollama directly in WSL
2. **Docker Ollama with GPU** - Run Ollama in a Docker container with GPU passthrough
3. **Docker Compose (CodexContinue)** - Use the project's Docker Compose configuration

Each approach has advantages, but you should only use one at a time to avoid port conflicts.

In [ ]:
def show_ollama_installation_options():
    display(Markdown("### Option 1: Standalone Ollama with GPU support"))
    display(Markdown("```bash\n# Install Ollama directly in WSL\ncurl -fsSL https://ollama.com/install.sh | sh\n\n# Start the Ollama service\nollama serve\n```"))
    
    display(Markdown("### Option 2: Docker Ollama with GPU support"))
    display(Markdown("```bash\n# Run Ollama in Docker with GPU support\ndocker run --rm -it --gpus=all -p 11434:11434 -v ollama:/root/.ollama ollama/ollama\n```"))
    
    display(Markdown("### Option 3: CodexContinue Docker Compose (Recommended)"))
    display(Markdown("```bash\n# First ensure no other Ollama instances are running\nsudo pkill -f ollama\ndocker stop $(docker ps -q --filter name=ollama)\n\n# Then start CodexContinue with GPU support\n./scripts/start-ollama-wsl.sh\n# Or use Docker Compose directly\ndocker compose -f docker-compose.yml up\n```"))

show_ollama_installation_options()

### Resolving Port Conflicts

The most common issue when starting Ollama is port conflicts. If you see an error like:

```
Error response from daemon: failed to set up container networking: driver failed programming external connectivity on endpoint codexcontinue-ollama-1: Bind for 0.0.0.0:11434 failed: port is already allocated
```

This means another process is already using port 11434. Follow these steps to resolve it:

In [ ]:
def show_port_conflict_fix():
    display(Markdown("### Fix for Port Conflicts"))
    
    # Create a function to detect what's using port 11434
    display(Markdown("Run these commands to identify and stop what's using port 11434:"))
    
    commands = [
        "# Find processes using port 11434",
        "sudo lsof -i :11434",
        "",
        "# Stop standalone Ollama if it's running",
        "sudo pkill -f ollama",
        "",
        "# Stop any Docker containers using the port",
        "docker stop $(docker ps -q --filter publish=11434)",
        "",
        "# Verify the port is now free",
        "sudo lsof -i :11434",
        "",
        "# Now try starting Ollama again",
        "./scripts/start-ollama-wsl.sh"
    ]
    
    display(Markdown("```bash\n" + "\n".join(commands) + "\n```"))
    
    # Create a manage-ollama-process.sh script suggestion
    display(Markdown("#### Helper Script"))
    display(Markdown("Here's a script you can create to manage Ollama processes:"))
    
    script_content = [
        "#!/bin/bash",
        "",
        "function stop_ollama() {",
        "  echo \"Stopping any running Ollama processes...\"",
        "  sudo pkill -f ollama 2>/dev/null",
        "  docker stop $(docker ps -q --filter name=ollama) 2>/dev/null",
        "  docker stop $(docker ps -q --filter publish=11434) 2>/dev/null",
        "  echo \"Done.\"",
        "}",
        "",
        "function check_port() {",
        "  echo \"Checking if port 11434 is in use...\"",
        "  if sudo lsof -i :11434 2>/dev/null; then",
        "    echo \"⚠️ Port 11434 is still in use.\"",
        "    return 1",
        "  else",
        "    echo \"✅ Port 11434 is free.\"",
        "    return 0",
        "  fi",
        "}",
        "",
        "function start_ollama_docker() {",
        "  echo \"Starting Ollama with Docker and GPU support...\"",
        "  stop_ollama",
        "  if check_port; then",
        "    ./scripts/start-ollama-wsl.sh",
        "  else",
        "    echo \"Cannot start Ollama until port 11434 is free.\"",
        "  fi",
        "}",
        "",
        "# Main script logic",
        "case \"$1\" in",
        "  stop)",
        "    stop_ollama",
        "    ;;",
        "  start)",
        "    start_ollama_docker",
        "    ;;",
        "  check)",
        "    check_port",
        "    ;;",
        "  *)",
        "    echo \"Usage: $0 {start|stop|check}\"",
        "    echo \"  start - Stop any running Ollama processes and start with Docker + GPU\"",
        "    echo \"  stop  - Stop all Ollama processes\"",
        "    echo \"  check - Check if port 11434 is available\"",
        "    exit 1",
        "esac",
        "",
        "exit 0"
    ]
    
    display(Markdown("```bash\n" + "\n".join(script_content) + "\n```"))
    display(Markdown("Save this to `scripts/manage-ollama-process.sh` and make it executable with `chmod +x scripts/manage-ollama-process.sh`."))
    display(Markdown("Then you can use it with:"))
    display(Markdown("```bash\n./scripts/manage-ollama-process.sh stop  # Stop all Ollama processes\n./scripts/manage-ollama-process.sh start # Start Ollama with Docker + GPU\n```"))

show_port_conflict_fix()

## Testing CodexContinue Application Ports

When running the full CodexContinue application, several services operate on different ports. Here's how to verify that all the necessary ports are available and working correctly.

In [ ]:
def check_port_availability(port):
    """Check if a port is available (not in use)."""
    import socket
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    try:
        s.bind(("127.0.0.1", port))
        display(Markdown(f"✅ Port {port} is available"))
        result = True
    except socket.error:
        display(Markdown(f"❌ Port {port} is already in use"))
        result = False
    finally:
        s.close()
    return result

def check_service_running(url, service_name):
    """Check if a service is running at the given URL."""
    import requests
    try:
        response = requests.get(url, timeout=2)
        if response.status_code < 400:
            display(Markdown(f"✅ {service_name} is running at {url}"))
            return True
        else:
            display(Markdown(f"⚠️ {service_name} at {url} returned status code {response.status_code}"))
            return False
    except requests.exceptions.RequestException as e:
        display(Markdown(f"❌ {service_name} at {url} is not accessible: {str(e)}"))
        return False

def check_codexcontinue_ports():
    """Check all ports used by CodexContinue services."""
    display(Markdown("### CodexContinue Port Status"))
    
    # Define the ports and services used by CodexContinue
    services = [
        {"port": 11434, "name": "Ollama API", "url": "http://localhost:11434/api/tags"},
        {"port": 8000, "name": "Backend API", "url": "http://localhost:8000/docs"},
        {"port": 8501, "name": "Streamlit Frontend", "url": "http://localhost:8501"},
        {"port": 5000, "name": "ML Service", "url": "http://localhost:5000/health"}
    ]
    
    # First check if the ports are in use
    display(Markdown("#### Port Availability Check"))
    for service in services:
        port_available = check_port_availability(service["port"])
        if port_available:
            display(Markdown(f"   - You will need to start the {service['name']} service"))
    
    # Ask user if they want to check running services
    display(Markdown("\n#### Service Connection Check"))
    check_services = input("Do you want to check if services are running? (y/n): ")
    
    if check_services.lower() == 'y':
        for service in services:
            check_service_running(service["url"], service["name"])
    else:
        display(Markdown("Skipping service connection check"))

# Run the check function
check_codexcontinue_ports()

### Resolving CodexContinue Port Conflicts

If you encounter port conflicts with any of the CodexContinue services, here's how to resolve them:

In [ ]:
def show_port_conflict_resolution():
    display(Markdown("#### Common Port Conflict Solutions"))
    
    # Table of ports and resolution methods
    port_table = [
        {"port": 11434, "service": "Ollama API", "resolution": "Use `scripts/manage-ollama-process.sh stop` to stop any running Ollama instances"},
        {"port": 8000, "service": "Backend API", "resolution": "Check for other FastAPI/Django/Flask services or modify the port in docker-compose.yml"},
        {"port": 8501, "service": "Streamlit Frontend", "resolution": "Check for other Streamlit apps or modify the port in docker-compose.yml"},
        {"port": 5000, "service": "ML Service", "resolution": "Check for other Flask/ML services or modify the port in docker-compose.yml"}
    ]
    
    # Create a Markdown table
    table_md = "| Port | Service | Resolution Method |\n| ---- | ------- | ---------------- |\n"
    for entry in port_table:
        table_md += f"| {entry['port']} | {entry['service']} | {entry['resolution']} |\n"
    
    display(Markdown(table_md))
    
    # Show how to check running processes on ports
    display(Markdown("#### How to check what's using a port"))
    display(Markdown("```bash\n# Replace PORT_NUMBER with the specific port (e.g., 8000)\nsudo lsof -i :PORT_NUMBER\n```"))
    
    # Show how to modify ports in docker-compose
    display(Markdown("#### How to modify service ports in docker-compose.yml"))
    docker_compose_example = """
```yaml
services:
  backend:
    ports:
      - "8000:8000"  # Change the first number to modify the host port
  
  frontend:
    ports:
      - "8501:8501"  # Change the first number to modify the host port
  
  ml-service:
    ports:
      - "5000:5000"  # Change the first number to modify the host port
```
    """
    display(Markdown(docker_compose_example))
    
    # Provide a script to test all services
    display(Markdown("#### Testing Script"))
    display(Markdown("Here's a bash script you can use to test if all CodexContinue services are running:"))
    test_script = """
```bash
#!/bin/bash
# test-codexcontinue-ports.sh - Test all CodexContinue services

echo "=== Testing CodexContinue Services ==="

# Define text colors
GREEN="\033[0;32m"
RED="\033[0;31m"
YELLOW="\033[0;33m"
NC="\033[0m" # No Color

# Function to check a service
check_service() {
  local url=$1
  local name=$2
  echo -n "Checking $name at $url: "
  if curl -s --head --request GET $url | grep -q "HTTP/";
  then
    echo -e "${GREEN}OK${NC}"
    return 0
  else
    echo -e "${RED}Failed${NC}"
    return 1
  fi
}

# Check Ollama
echo "\nTesting Ollama API..."
if check_service "http://localhost:11434/api/tags" "Ollama API"; then
  # Try a quick model query
  echo "Testing Ollama model response..."
  curl -s -X POST http://localhost:11434/api/generate -d '{"model":"llama3","prompt":"hello","stream":false}' | grep -q 'response' && echo -e "${GREEN}Model response OK${NC}" || echo -e "${RED}Model response failed${NC}"
fi

# Check Backend API
echo "\nTesting Backend API..."
check_service "http://localhost:8000/docs" "API Documentation"

# Check Streamlit Frontend
echo "\nTesting Streamlit Frontend..."
check_service "http://localhost:8501" "Web Interface"

# Check ML Service
echo "\nTesting ML Service..."
check_service "http://localhost:5000/health" "ML Service Health"

# Final summary
echo "\n=== Test Summary ==="
echo "To start all services: ./scripts/start-codexcontinue.sh"
echo "To check Ollama: ./scripts/manage-ollama-process.sh status"
```
    """
    display(Markdown(test_script))

show_port_conflict_resolution()

### Creating a Port Testing Script for CodexContinue

Let's create a reusable bash script for testing all CodexContinue service ports that you can run anytime.

In [ ]:
def create_port_testing_script():
    # Create the script content
    script_content = """#!/bin/bash
# test-codexcontinue-ports.sh - Test all CodexContinue service ports

# Colors for output
GREEN='\033[0;32m'
RED='\033[0;31m'
YELLOW='\033[0;33m'
BLUE='\033[0;34m'
NC='\033[0m' # No Color

# Helper functions
log() {
    echo -e "${GREEN}[INFO]${NC} $1"
}

warn() {
    echo -e "${YELLOW}[WARNING]${NC} $1"
}

error() {
    echo -e "${RED}[ERROR]${NC} $1"
}

section() {
    echo -e "\n${BLUE}=== $1 ===${NC}"
}

# Make sure we're in the project root
cd "$(dirname \"$0\")/.."

echo "==============================================="
echo "CodexContinue Service Port Test"
echo "==============================================="
echo "This script tests all service ports used by CodexContinue"
echo

# Define the services and their ports
declare -A services=(
    ["Ollama API"]=11434
    ["Backend API"]=8000
    ["Streamlit Frontend"]=8501
    ["ML Service"]=5000
)

# Function to check if a port is in use
check_port_in_use() {
    local port=$1
    local service_name=$2
    
    if lsof -i :"$port" &>/dev/null; then
        log "Port $port ($service_name) is in use"
        return 0 # true, port is in use
    else
        warn "Port $port ($service_name) is not in use - service may not be running"
        return 1 # false, port is not in use
    fi
}

# Function to check if a service is responding
check_service_responding() {
    local port=$1
    local service_name=$2
    local endpoint=$3
    
    # Create URL from port and endpoint
    local url="http://localhost:${port}${endpoint}"
    
    echo -n "  Testing $service_name at $url: "
    
    # Try to connect to the service
    if curl -s --head --request GET "$url" -m 2 | grep -q "HTTP/";
    then
        echo -e "${GREEN}OK${NC}"
        return 0
    else
        echo -e "${RED}Failed${NC}"
        return 1
    fi
}

# Check all port usage first
section "Checking Port Usage"
for service in "${!services[@]}"; do
    port=${services[$service]}
    check_port_in_use "$port" "$service"
done

# Now check service responses
section "Testing Service Responses"

# Ollama API
if check_port_in_use "${services['Ollama API']}" "Ollama API"; then
    check_service_responding "${services['Ollama API']}" "Ollama API" "/api/tags"
    
    # Try a quick model query if available
    echo "  Testing Ollama model response..."
    if curl -s -X POST http://localhost:11434/api/generate -d '{"model":"llama3","prompt":"hello","stream":false}' | grep -q 'response'; then
        log "Ollama model response: OK"
    else
        warn "Ollama model response failed - models may not be loaded"
    fi
fi

# Backend API
if check_port_in_use "${services['Backend API']}" "Backend API"; then
    check_service_responding "${services['Backend API']}" "Backend API" "/docs"
fi

# Streamlit Frontend
if check_port_in_use "${services['Streamlit Frontend']}" "Streamlit Frontend"; then
    check_service_responding "${services['Streamlit Frontend']}" "Streamlit Frontend" "/"
fi

# ML Service
if check_port_in_use "${services['ML Service']}" "ML Service"; then
    check_service_responding "${services['ML Service']}" "ML Service" "/health"
fi

# Final summary and tips
section "Summary and Tips"
echo "If any services are not running, try the following:"
echo
echo "1. To start all CodexContinue services:"
echo "   ./scripts/start-codexcontinue.sh"
echo
echo "2. To manage Ollama separately:"
echo "   ./scripts/manage-ollama-process.sh stop    # Stop Ollama"
echo "   ./scripts/manage-ollama-process.sh start   # Start Ollama"
echo "   ./scripts/manage-ollama-process.sh status  # Check Ollama status"
echo
echo "3. For port conflicts, check what processes are using the ports:"
echo "   sudo lsof -i :PORT_NUMBER  # Replace PORT_NUMBER with the port"
echo
echo "4. Modify ports in docker-compose.yml if needed (change first number):"
echo "   ports:"
echo "     - \"NEW_PORT:ORIGINAL_PORT\""

echo
echo "For more information, see docs/troubleshooting/"
"""
    
    # Ask for confirmation
    create_script = input("Do you want to create the port testing script in the scripts directory? (y/n): ")
    
    if create_script.lower() == 'y':
        # Define the path
        script_path = "../scripts/test-codexcontinue-ports.sh"
        
        try:
            with open(script_path, 'w') as f:
                f.write(script_content)
            
            # Make the script executable
            import os
            os.chmod(script_path, 0o755)
            
            display(Markdown(f"✅ Created port testing script at: {script_path}"))
            display(Markdown("To use it, run:\n```bash\n./scripts/test-codexcontinue-ports.sh\n```"))
            
            # Suggest adding to git
            display(Markdown("Don't forget to add it to git:\n```bash\ngit add scripts/test-codexcontinue-ports.sh\ngit commit -m "Add port testing script for CodexContinue services"\n```"))
            
            return True
        except Exception as e:
            display(Markdown(f"❌ Error creating script: {str(e)}"))
            return False
    else:
        display(Markdown("Script creation skipped."))
        # Show the script content instead
        display(Markdown("Here's the script content if you want to create it manually later:"))
        display(Markdown("```bash\n" + script_content + "\n```"))
        return False

create_port_testing_script()